In [240]:
import pennylane as qml
import cv2
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import glob
from tqdm import tqdm
import pandas as pd

In [241]:
import matplotlib.pyplot as plt

In [242]:
np.random.seed(420)

In [243]:
#Lets load in our training data. We got this data from the Pytorch CNN tutorial. It is images of dogs and cats grayscale with labels accordingly
training_data = np.load("training_data.npy", allow_pickle=True)
print(training_data)

[[array([[ 45,  86,  91, ..., 161, 204, 187],
       [ 43,  67,  70, ..., 189, 185, 197],
       [ 34,  60,  54, ..., 195, 175, 181],
       ...,
       [132, 137, 143, ..., 188, 189, 132],
       [130, 130, 136, ..., 185, 193, 188],
       [126, 132, 136, ..., 190, 188, 189]], dtype=uint8)
  array([1., 0.])]
 [array([[122, 119, 107, ...,  60,  64,  73],
       [121, 119, 114, ...,  82,  97,  87],
       [119, 121, 118, ...,  93,   7,   9],
       ...,
       [ 76, 112, 118, ..., 105,  99,  90],
       [ 80,  92, 106, ...,  94,  88,  79],
       [ 61,  83,  92, ...,  80,  74,  60]], dtype=uint8)
  array([1., 0.])]
 [array([[109, 180, 173, ...,  65,  65,  54],
       [183, 170, 180, ...,  64,  63,  54],
       [171, 164, 182, ...,  67,  61,  53],
       ...,
       [144, 150, 145, ..., 124, 118, 112],
       [146, 145, 164, ..., 137, 119,  97],
       [117, 142, 170, ..., 121, 118, 109]], dtype=uint8)
  array([1., 0.])]
 ...
 [array([[102,  93, 130, ...,  71,  50,  78],
       [102, 101

In [244]:
#Lets separate out the images and the labels
images = []
labels = [] 

for i in training_data:
    images.append(i[0])
    labels.append(i[1])
print(images[1:10])
print(labels)

[array([[122, 119, 107, ...,  60,  64,  73],
       [121, 119, 114, ...,  82,  97,  87],
       [119, 121, 118, ...,  93,   7,   9],
       ...,
       [ 76, 112, 118, ..., 105,  99,  90],
       [ 80,  92, 106, ...,  94,  88,  79],
       [ 61,  83,  92, ...,  80,  74,  60]], dtype=uint8), array([[109, 180, 173, ...,  65,  65,  54],
       [183, 170, 180, ...,  64,  63,  54],
       [171, 164, 182, ...,  67,  61,  53],
       ...,
       [144, 150, 145, ..., 124, 118, 112],
       [146, 145, 164, ..., 137, 119,  97],
       [117, 142, 170, ..., 121, 118, 109]], dtype=uint8), array([[ 19,  16,  18, ...,  14,  15,  15],
       [ 22,  21,  19, ...,  14,  15,  15],
       [ 23,  16,  23, ...,  14,  16,  15],
       ...,
       [166, 167, 171, ..., 156, 153, 145],
       [175, 172, 167, ..., 149, 128, 145],
       [172, 165, 166, ..., 162, 157, 155]], dtype=uint8), array([[253, 253, 253, ..., 251, 249, 250],
       [253, 253, 254, ...,  11,   7,   5],
       [ 17,  13,  14, ...,  17,  15, 

In [245]:
#Split data into train and test datasets
train_images = images[0:1000]
test_images = images[1001:1200]

train_labels = labels[0:1000]
test_labels = labels[1001:1200]

In [246]:
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T

In [247]:
label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = [label_0, label_1]

In [248]:
dev = qml.device("default.qubit", wires=1)

In [249]:
@qml.qnode(dev)
def qcircuit(params, x=None, y=None):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for p in params:
        qml.Rot(*x, wires=0)
        qml.Rot(*p, wires=0)
    return qml.expval(qml.Hermitian(y, wires=[0]))

In [250]:
def cost(params, x, y, state_labels=None):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): array of state representations for labels

    Returns:
        float: loss value to be minimized
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    if np.argmax(y) == 1:
        y_use = dm_labels[1]
    elif np.argmax(y) == 0:
        y_use = dm_labels[0]
    for i in range(len(x)):
        for j in range(len(x[i])):
            f = qcircuit(params, x=[x[i][j][0],x[i][j][1],x[i][j][2]], y=y_use)
            loss = loss + (1 - f) ** 2
    return loss / (len(x)*len(x[i]))

In [251]:
def test(params, x, y, state_labels=None):

    #Tests on a given set of data.

    #Args:
    #    params (array[float]): array of parameters
    #    x (array[float]): 2-d array of input vectors
    #    y (array[float]): 1-d array of targets
    #    state_labels (array[float]): 1-d array of state representations for labels

    #Returns:
    #    predicted (array([int]): predicted labels for test data
    #    output_states (array[float]): output quantum states from the circuit

    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        for j in range(len(x[i])):
            fidel_function = lambda y: qcircuit(params, x=[x[i][j][0],x[i][j][1],x[i][j][2]], y=y)
            fidelities = [fidel_function(dm) for dm in dm_labels]
            best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(pd.get_dummies(predicted)), np.array(fidelity_values)

In [252]:
def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    score = 0
    print(y_pred)
    print(y_true)
    for i in range(len(y_true)):
        if np.argmax(y_true[i]) == np.argmax(y_pred[i]):
            score += 1 
    print(score/len(y_true))
    return (score / len(y_true))

In [253]:
def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [254]:
# Generate training and test data
num_training = 200
num_test = 2000

# Train using Adam optimizer and evaluate the classifier
num_layers = 3
learning_rate = 0.1
epochs = 10
batch_size = 32

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

# initialize random weights
params = np.random.uniform(size=(num_layers, 3))

predicted_train, fidel_train = test(params, np.array(train_images), np.array(train_labels), state_labels)
accuracy_train = accuracy_score(np.array(train_labels), predicted_train)

predicted_test, fidel_test = test(params, np.array(test_images), np.array(test_labels), state_labels)
accuracy_test = accuracy_score(np.array(test_labels), predicted_test)

# save predictions with random weights for comparison
initial_predictions = predicted_test

loss = cost(params, np.array(train_images), np.array(train_labels), state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test
    )
)

for it in range(epochs):
    for Xbatch, ybatch in tqdm(iterate_minibatches(np.array(train_images), np.array(train_labels), batch_size=batch_size)):
        params = opt.step(lambda v: cost(v, Xbatch, ybatch, state_labels), params)

    predicted_train, fidel_train = test(params, np.array(train_images), np.array(train_labels), state_labels)
    accuracy_train = accuracy_score(np.array(train_labels), predicted_train)
    loss = cost(params, np.array(train_images), np.array(train_labels), state_labels)

    predicted_test, fidel_test = test(params, np.array(test_images), np.array(test_labels), state_labels)
    accuracy_test = accuracy_score(np.array(test_labels), predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )
    print(predicted_test)

[[0 1]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
0.508
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 














0it [00:00, ?it/s]

Epoch:  0 | Cost: 229.969385 | Train accuracy: 0.508000 | Test Accuracy: 0.482412















1it [00:24, 24.89s/it]












2it [00:49, 24.82s/it]












3it [01:14, 24.77s/it]












4it [01:38, 24.76s/it]












5it [02:03, 24.73s/it]












6it [02:28, 24.72s/it]












7it [02:52, 24.66s/it]












8it [03:17, 24.65s/it]












9it [03:42, 24.76s/it]












10it [04:07, 24.81s/it]












11it [04:32, 24.85s/it]












12it [04:57, 24.79s/it]












13it [05:22, 24.91s/it]












14it [05:47, 24.96s/it]












15it [06:12, 24.97s/it]












16it [06:37, 24.94s/it]












17it [07:02, 24.99s/it]












18it [07:27, 24.96s/it]












19it [07:51, 24.90s/it]












20it [08:16, 24.89s/it]












21it [08:41, 24.86s/it]












22it [09:06, 24.89s/it]












23it [09:31, 24.79s/it]












24it [09:55, 24.71s/it]












25it [10:20, 24.68s/it]












26it [10:44, 24.68s/it]












27it [11:09, 24.67s/it]












28it [11:34, 24.66s/it]


[[0 1]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
0.511


...

[[0 1]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
0.508















0it [00:00, ?it/s]

[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]














1it [00:24, 24.57s/it]












2it [00:49, 24.72s/it]












3it [01:14, 24.70s/it]












4it [01:38, 24.61s/it]












5it [02:03, 24.54s/it]












6it [02:27, 24.58s/it]












7it [02:52, 24.57s/it]












8it [03:16, 24.53s/it]












9it [03:41, 24.52s/it]












10it [04:05, 24.57s/it]












11it [04:30, 24.56s/it]












12it [04:54, 24.54s/it]












13it [05:19, 24.54s/it]












14it [05:43, 24.50s/it]












15it [06:08, 24.51s/it]












16it [06:32, 24.51s/it]












17it [06:57, 24.52s/it]












18it [07:22, 24.56s/it]












19it [07:46, 24.55s/it]

KeyboardInterrupt: 